## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


확률적 경사 하강법, 미니배치 경사 하강법

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

학습률이 너무 높기 때문이다. 학습률을 낮춘다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

높은 편향이 문제다. 하이퍼파라미터를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

* 평범한 선형회귀보다는 규제가 있는 모델의 성능이 일반적으로 좋기 때문이다.

* 특성이 몇 개뿐일 것 같을때.

* 특성끼리 강하게 연관된 경우에 적합.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [3]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

iris = datasets.load_iris()

X = iris["data"][:, (2,3)]
y = iris['target']

In [5]:
# 원-핫 인코딩
num_classes = len(np.unique(y))
y_onehot = np.eye(num_classes)[y]

X = (X - X.mean(axis=0)) / X.std(axis=0)

np.random.seed(42)
indices = np.random.permutation(len(X))
train_size = int(0.8 * len(X))
train_idx, val_idx = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_idx], y_onehot[train_idx]
X_val, y_val = X[val_idx], y_onehot[val_idx]

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-9)) / m

n_features = X.shape[1]
W = np.zeros((n_features, num_classes))
b = np.zeros((1, num_classes))

learning_rate = 0.1
max_epochs = 5000
patience = 50
best_val_loss = np.inf
patience_counter = 0

train_losses, val_losses = [], []

# 배치 경사 하강법 학습
for epoch in range(max_epochs):
    logits = X_train.dot(W) + b
    y_pred = softmax(logits)
    train_loss = compute_loss(y_train, y_pred)
    grad_W = (1 / X_train.shape[0]) * X_train.T.dot(y_pred - y_train)
    grad_b = (1 / X_train.shape[0]) * np.sum(y_pred - y_train, axis=0, keepdims=True)
    W -= learning_rate * grad_W
    b -= learning_rate * grad_b

    val_pred = softmax(X_val.dot(W) + b)
    val_loss = compute_loss(y_val, val_pred)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # 조기 종료
    if val_loss < best_val_loss - 1e-5:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

In [8]:
## 정확도
y_train_pred = np.argmax(softmax(X_train.dot(W)+b), axis=1)
y_val_pred = np.argmax(softmax(X_val.dot(W)+b), axis=1)
train_acc = np.mean(y_train_pred == np.argmax(y_train, axis=1))
val_acc = np.mean(y_val_pred == np.argmax(y_val, axis=1))
print(train_acc, val_acc)

0.9583333333333334 0.9666666666666667
